# Importante 

* Tener en cuenta bajadas de Procedimientos, Personas Armas, Divisas , vehiculos (secuestrados, ministerio), Narcotrafico general,Objetos y pegar en la carpeta bajadas de fecha del primero del 1 del mes en informar hasta fecha del informe 

### IMPORTACIÓN DE LIBREARIAS


In [2]:
import pandas as pd
from  Parametros import * 
from  Funciones import * 
from openpyxl import load_workbook


### ACCEDER BASE DE DATOS

In [4]:
df_base_procedimientos = pd.read_excel("db/Base_procedimientos_2023_2024.xlsx")
df_base_operaciones = pd.read_excel("db/Base_operaciones_2023_2024.xlsx")


### PROCEDIMIENTOS GENERALES




In [ ]:
excel_bajada_procedimientos_generales = filtrar_procedimientos_generales("bajadas/bajada_general.xls")

df_geog = pd.DataFrame()
df_geog['UID'] = excel_bajada_procedimientos_generales.apply(generar_uid_sigpol, axis=1)
df_geog['ID_OPERATIVO'] = excel_bajada_procedimientos_generales.apply(procesar_causa_judicial, axis=1)
df_geog['FUERZA_INTERVINIENTE'] = "PSA"
df_geog['ID_PROCEDIMIENTO'] = df_geog['UID']
df_geog['CAUSAJUDICIALNUMERO'] = excel_bajada_procedimientos_generales['CAUSAJUDICIALNUMERO'].copy()
df_geog['UNIDAD_INTERVINIENTE'] = excel_bajada_procedimientos_generales['UOSP']
df_geog['DESCRIPCIÓN'] = excel_bajada_procedimientos_generales.apply(procesar_descripcion, axis=1)
df_geog['TIPO_INTERVENCION'] = excel_bajada_procedimientos_generales.apply(procesar_tipo_procedimiento, axis=1)
df_geog['PROVINCIA'] = excel_bajada_procedimientos_generales.apply(procesar_provincia, axis=1)
df_geog['DEPARTAMENTO O PARTIDO'] = excel_bajada_procedimientos_generales.apply(procesar_municipio, axis=1)
df_geog['LOCALIDAD'] = "-"
df_geog['DIRECCION'] = excel_bajada_procedimientos_generales.apply(procesar_direccion, axis=1)
df_geog['LATITUD'] = excel_bajada_procedimientos_generales['GEOREFERENCIA_Y']
df_geog['LONGITUD'] = excel_bajada_procedimientos_generales['GEOREFERENCIA_X']
df_geog['FECHA'] = pd.to_datetime(excel_bajada_procedimientos_generales['DENUNCIAFECHA'], errors='coerce').dt.strftime('%d/%m/%Y')
df_geog['HORA'] = pd.to_datetime(excel_bajada_procedimientos_generales['DENUNCIAFECHA'], errors='coerce').dt.strftime('%H:%M')
df_geog['ZONA_SEGURIDAD_FRONTERAS'] = "-"
df_geog['PASO_FRONTERIZO'] = "-"
df_geog['OTRAS AGENCIAS INTERVINIENTES'] = "-"
df_geog['Observaciones - Detalles'] = "-"

df_geog = df_geog[[ "UID",'FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO',
                                     'UNIDAD_INTERVINIENTE', 'DESCRIPCIÓN', 'TIPO_INTERVENCION',
                                     'PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION',
                                     'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD',
                                     'FECHA', 'HORA', 'OTRAS AGENCIAS INTERVINIENTES', 'Observaciones - Detalles']]


df_geog_final = df_geog[~df_geog['UID'].isin(set(df_base_procedimientos['UID']))]
print(f"\nCasos ya informados: {len(df_base_procedimientos)}")
print(f"Casos nuevos para informar : {len(df_geog_final)}")



Casos ya informados: 8738
Casos nuevos para informar : 51


### HOJA PERSONAS

In [ ]:
import pandas as pd

# Paso 1: Cargar datos iniciales
excel_bajada_personas = pd.read_excel("bajadas/bajada_general_persona.xls")
print(f"Total de registros cargados: {len(excel_bajada_personas)}")

# Paso 2: Generar UID
excel_bajada_personas['UID'] = excel_bajada_personas.apply(generar_uid_sigpol, axis=1)

# Paso 3: Coincidencia de UID entre excel_bajada_personas y df_geog_final
uid_coincidentes = excel_bajada_personas[excel_bajada_personas['UID'].isin(df_geog_final['UID'])]
print(f"Cantidad de registros con UID coincidente: {len(uid_coincidentes)}")

# Crear DataFrame de detenidos y aprendidos
df_detenidos_aprendidos = pd.DataFrame()
df_detenidos_aprendidos['UID'] = uid_coincidentes['UID']
df_detenidos_aprendidos['EDAD'] = uid_coincidentes.apply(procesar_edad, axis=1)
df_detenidos_aprendidos['SEXO'] = uid_coincidentes.apply(procesar_sexo, axis=1)
df_detenidos_aprendidos['GENERO'] = uid_coincidentes.apply(procesar_genero, axis=1)
df_detenidos_aprendidos['NACIONALIDAD'] = uid_coincidentes.apply(procesar_nacionalidad, axis=1)
df_detenidos_aprendidos['SITUACION_PROCESAL'] = uid_coincidentes.apply(procesar_situacion_judicial, axis=1)
df_detenidos_aprendidos['DELITO_IMPUTADO'] = uid_coincidentes.apply(procesar_tipo_delito, axis=1)
df_detenidos_aprendidos['JUZGADO_INTERVINIENTE'] = uid_coincidentes.apply(procesar_juzgado, axis=1)
df_detenidos_aprendidos['CARATULA_CAUSA'] = uid_coincidentes.apply(procesar_caratula, axis=1)

print(f"Total de registros después del procesamiento de columnas específicas: {len(df_detenidos_aprendidos)}")

# Paso 4: Filtrar SITUACION_PROCESAL 'NO INFORMAR'
df_detenidos_aprendidos = df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] != 'NO INFORMAR']
print(f"Cantidad tras exclusión de 'NO INFORMAR': {len(df_detenidos_aprendidos)}")

# Paso 5: Fusionar df_geog_final y df_detenidos_aprendidos en base al UID
df_detenidos_aprendidos = pd.merge(df_geog_final, df_detenidos_aprendidos, on='UID', how='left')
print(f"Total de registros después de la fusión con df_geog_final: {len(df_detenidos_aprendidos)}")

# Paso 6: Relleno de valores vacíos con '-'
df_detenidos_aprendidos = df_detenidos_aprendidos.apply(lambda x: x.fillna('-') if x.dtype == 'object' else x)


# Paso 7: Selección y reordenamiento de columnas
df_detenidos_aprendidos_final = df_detenidos_aprendidos[[
    "UID",
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    'EDAD',
    'SEXO',
    'GENERO',
    'NACIONALIDAD',
    'SITUACION_PROCESAL',
    "DELITO_IMPUTADO",
    'JUZGADO_INTERVINIENTE',
    'CARATULA_CAUSA'
]]
print(f"Total de registros después de seleccionar y reordenar columnas: {len(df_detenidos_aprendidos_final)}")

# Paso 8: Verificación final de cantidades por situación procesal
print(f"Cantidad total de partes procesadas: {len(excel_bajada_personas)}")
print(f"Cantidad de detenidos: {len(df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] == 'DETENIDO'])}")
print(f"Cantidad de liberados: {len(df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] == 'LIBERADO'])}")
print(f"Cantidad de víctimas: {len(df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] == 'VICTIMA'])}")


Total de registros cargados: 533
Cantidad de registros con UID coincidente: 32
Total de registros después del procesamiento de columnas específicas: 32
Cantidad tras exclusión de 'NO INFORMAR': 28
Total de registros después de la fusión con df_geog_final: 51
Total de registros después de seleccionar y reordenar columnas: 51
Cantidad total de partes procesadas: 533
Cantidad de detenidos: 0
Cantidad de liberados: 11
Cantidad de víctimas: 17


### HOJA OTROS DELITOS

In [8]:
# Crear un nuevo dataframe para otro delitos
df_otros_delitos = df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] == 'VICTIMA']
# Filtro los que son victima de detenidos y aprendidos
df_detenidos_aprendidos = df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] != 'VICTIMA']

# Cambiar el nombre de algunas columnas en el DataFrame de víctimas
df_otros_delitos = df_otros_delitos.rename(columns={
    'DELITO_IMPUTADO': 'TIPO_OTRO_DELITO',
    'GENERO': 'GENERO_VICTIMA',
    'EDAD': 'EDAD_VICTIMA',
})

df_otros_delitos["OBSERVACIONES"] ="-"
# Seleccionar las columnas deseadas en el DataFrame de víctimas
df_otros_delitos_final = df_otros_delitos[[
    "UID",
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    "TIPO_OTRO_DELITO",
    'GENERO_VICTIMA',
    'EDAD_VICTIMA',
    'NACIONALIDAD',
    'OBSERVACIONES'
]]


# Imprimir la cantidad de registros en el DataFrame de otros delitos para verificar
print(f"Cantidad de registros en df_otros_delitos: {len(df_otros_delitos)}")


Cantidad de registros en df_otros_delitos: 0


### INCAUTACIONES

# OPERACIONES

### ORDEN DE SERVICIOS

In [ ]:
excel_bajada_operaciones = load_workbook("bajadas/operaciones.xlsx")
hoja_geog = excel_bajada_operaciones['GEOG. PROCEDIMIENTO']

df_operaciones = leer_excel_a_df(hoja_geog)
df_operaciones = df_operaciones.dropna(subset=['ID_PROCEDIMIENTO'], how='any').copy()
df_operaciones["UID"] = df_operaciones.apply(generar_uid_operaciones , axis=1)
df_operaciones['ID_PROCEDIMIENTO'] = df_operaciones['ID_PROCEDIMIENTO'].apply(colocar_guion_espacio)
df_operaciones['UNIDAD_INTERVINIENTE'] = df_operaciones.apply(procesar_unidad, axis=1)
df_operaciones['ID_OPERATIVO'] = df_operaciones['ID_PROCEDIMIENTO'].copy()
df_operaciones['FECHA'] = pd.to_datetime(df_operaciones['FECHA'], errors='coerce')

# Aplicar la función para identificar las filas que son diferentes

df_operaciones = colocar_contador(df_operaciones , df_base_operaciones)